**Improvements**
1. Searches Vitals PDF for normal ranges 
    1. Removed explicit vital sign information in system prompt




In [1]:
#Imports
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext
import os
import threading 
import time
from sqlalchemy import create_engine, Column, Integer, String, Text, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship, declarative_base
from sqlalchemy.exc import SQLAlchemyError
import atexit

# Function to handle chatbot summarization
from langchain_community.document_loaders import JSONLoader, PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter, RecursiveJsonSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import OpenAIEmbeddings
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.retrievers import MergerRetriever
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
#Chat History Database setup
DATABASE_URL = "sqlite:///Nurse_Chat_History_2.db"
Base = declarative_base()

class Session(Base):
    __tablename__ = "sessions"
    id = Column(Integer, primary_key=True)
    session_id = Column(String, unique=True, nullable=False)
    messages = relationship("Message", back_populates="session")

class Message(Base):
    __tablename__ = "messages"
    id = Column(Integer, primary_key=True)
    session_id = Column(Integer, ForeignKey("sessions.id"), nullable=False)
    role = Column(String, nullable=False)
    content = Column(Text, nullable=False)
    session = relationship("Session", back_populates="messages")

# Create the database and the tables
engine = create_engine(DATABASE_URL)
Base.metadata.create_all(engine)
SessionLocal = sessionmaker(bind=engine)

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


In [3]:
# Function to save a single message
def save_message(session_id: str, role: str, content: str):
    with SessionLocal() as db:
        try:
            session = db.query(Session).filter(Session.session_id == session_id).first()
            if not session:
                session = Session(session_id=session_id)
                db.add(session)
                db.commit()
                db.refresh(session)

            db.add(Message(session_id=session.id, role=role, content=content))
            db.commit()
        except SQLAlchemyError:
            db.rollback()


# Function to load chat history
def load_session_history(session_id: str) -> ChatMessageHistory:
    chat_history = ChatMessageHistory()
    with SessionLocal() as db:
        try:
            session = db.query(Session).filter(Session.session_id == session_id).first()
            if session:
                for message in session.messages:
                    chat_history.add_message({"role": message.role, "content": message.content})
        except SQLAlchemyError:
            pass

    return chat_history


# Modify the get_session_history function to use the database
def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        # Load from the database if not in store
        store[session_id] = load_session_history(session_id)
    return store[session_id]


# Ensure you save the chat history to the database when needed
def save_all_sessions():
    for session_id, chat_history in store.items():
        for message in chat_history.messages:
            save_message(session_id, message["role"], message["content"])

# Register function to save sessions before exit
atexit.register(save_all_sessions)

store = {}


In [ ]:
def Nurse2NurseChatbotSummarize(filepath: str, session_id: str, promptQuestion: str):
    try:
        llm = ChatOpenAI(model='gpt-4o')
        
        #initilze JSONLoader to load in JSON data. jq_schemea set to '.' to load in entire JSON, text_content = False as to not parse through text
        loader = JSONLoader(file_path=filepath, jq_schema='.', text_content=False)

        #Load JSON file into memory --> docs is a list of documents  
        docs = loader.load()

        #Instantiate the recursive text splitter. Recursively break loaded document into chunks of up to 5000 characters with 200 characters of overlap. add_start_index = True to keep track of the position of each chunk
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=200, add_start_index=True)  # Increased chunk-size to 5000 from 1000

        #Apply text_splitter to loaded document and break JSON into smaller documents. splits is a list of smaller chunks of text
        splits = text_splitter.split_documents(docs)

        #generate vector embeddings for each document
        vectorstore = InMemoryVectorStore.from_documents(documents=splits, embedding=OpenAIEmbeddings())

        #instantiate retriever configured to use similarity search to find documents in vector store. Used to provide context to LLM
        retriever = vectorstore.as_retriever(search_type='similarity', search_kwargs={'k': 4})

        pdf_loader = PyPDFLoader(file_path='NormalVitals.pdf')
        pdf_docs = pdf_loader.load()
        pdf_splits = text_splitter.split_documents(pdf_docs)
        pdf_vectorstore = InMemoryVectorStore.from_documents(documents=pdf_splits, embedding=OpenAIEmbeddings())
        pdf_retriever = pdf_vectorstore.as_retriever(search_type='similarity', search_kwargs={'k': 4})


        #Above lines document processing pipeline: Load JSON, split content into smaller chunks, embed chunks, similarity-based retrieval

        combined_retriever = MergerRetriever(retrievers = [retriever, pdf_retriever], retriever_weights = [0.5,0.5])

        contextualize_q_system_prompt = (
            "Given a chat history and the latest user question "
            "which might reference context in the chat history, "
            "formulate a standalone question which can be understood "
            "without the chat history. Do NOT answer the question, "
            "just reformulate it if needed and otherwise return it as is."
        )

        contextualize_q_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", contextualize_q_system_prompt),
                MessagesPlaceholder("chat_history"),
                ("human", "{input}"),
            ]
        )

        history_aware_retriever = create_history_aware_retriever(llm, combined_retriever, contextualize_q_prompt)

        system_prompt = (
            "You are a nurse in the NICU at the end of your shift, preparing for patient handover. "
            "Provide the incoming nurse with all pertinent information for their shift. "
            "Use the 'Normal Vital Signs in Infants, Children, and Adolescents' context to determine if the vital signs are within normal ranges."
            "Use technical terms such as tachycardia, bradycardia, tachypnea, and bradypnea. "
            "IF the vital sign is 141-170 and the vitalsign is measured to be 149, that therefore means within normal range."
            "Indicate if vital signs are above (tachycardia/tachypnea) or below (bradycardia/bradypnea) normal ranges. "
            "Summarize the patient's status in 3 sentences, including interventions performed and the range of vital signs observed. "
            "The summary should be technical and assume the colleague understands medical terminology. "
            "Do not explain what is normal; focus on the context as your knowledge. "
            "Avoid diagnosing. "
            "\n\n{context}"
        )

        qa_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                MessagesPlaceholder("chat_history"),
                ("human", "{input}"),
            ]
        )
        question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
        rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

        conversational_rag_chain = RunnableWithMessageHistory(
            rag_chain,
            get_session_history,
            input_messages_key="input",
            history_messages_key="chat_history",
            output_messages_key="answer",
        )
        response = conversational_rag_chain.invoke(
            {"input": promptQuestion},
            config={
                "configurable": {"session_id": session_id}
            },  # constructs a key "abc123" in `store`.
        )

        # Save the user question and AI response to the database
        save_message(session_id, "human", promptQuestion)
        save_message(session_id, "ai", response['answer'])

        return response['answer']
    except Exception as e:
        return f"Error: {str(e)}"


ImportError: cannot import name 'create_stuff_documents_chain' from 'langchain.chains' (c:\Users\elfo\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain\chains\__init__.py)

In [5]:
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext
import threading
import time
import os
from tkinterdnd2 import TkinterDnD, DND_FILES

# Global loading flag
loading = False

# Colors and Styles
BG_COLOR = "#2C2C2C"
FG_COLOR = "#FFFFFF"
BTN_COLOR = "#4CAF50"
ENTRY_COLOR = "#3E3E3E"
FONT = ("Arial", 12)

# Loading Dots Animation
def loading_dots():
    dot_count = 0
    while loading:
        dot_count = (dot_count % 3) + 1
        dots = ". " * dot_count
        chat_history_text.configure(state='normal')
        chat_history_text.delete('end-1c linestart', 'end-1c')  # Remove previous dots
        chat_history_text.insert(tk.END, dots)
        chat_history_text.configure(state='disabled')
        root.update_idletasks()
        time.sleep(0.5)

# Run Chatbot Function
def run_chatbot():
    filepath = file_entry.get()
    session_id = session_entry.get()
    prompt = prompt_entry.get()

    if not os.path.exists(filepath):
        messagebox.showerror("Error", "The selected file does not exist.")
        return

    if not session_id:
        messagebox.showerror("Error", "Please enter a valid session ID.")
        return

    if not prompt:
        messagebox.showerror("Error", "Please enter a prompt.")
        return

    chat_history_text.configure(state='normal')
    chat_history_text.insert(tk.END, f"You: {prompt}\nBot: ")
    chat_history_text.configure(state='disabled')
    root.update_idletasks()

    global loading
    loading = True
    threading.Thread(target=loading_dots, daemon=True).start()

    def generate_response():
        global loading
        result = Nurse2NurseChatbotSummarize(filepath, session_id, prompt)  # Placeholder for actual function
        loading = False
        chat_history_text.configure(state='normal')
        chat_history_text.delete('end-1c linestart', 'end-1c')  # Remove loading dots
        chat_history_text.insert(tk.END, f"{result}\n\n")
        chat_history_text.configure(state='disabled')
        prompt_entry.delete(0, tk.END)

    threading.Thread(target=generate_response, daemon=True).start()

# End Chat Function
def end_chat():
    chat_history_text.configure(state='normal')
    chat_history_text.insert(tk.END, "Chat session ended.\n\n")
    chat_history_text.configure(state='disabled')
    session_entry.delete(0, tk.END)
    prompt_entry.delete(0, tk.END)
    file_entry.delete(0, tk.END)
    root.after(1000, root.destroy)  # Close the GUI after 1 second

# Drag-and-Drop Handler
def on_file_drop(event):
    file_entry.delete(0, tk.END)  # Clear existing text
    file_entry.insert(0, event.data.strip())  # Insert the dropped file path

# GUI Setup using TkinterDnD
root = TkinterDnD.Tk()
root.title("Chatbot GUI for JSON Summarization")
root.geometry("800x600")
root.configure(bg=BG_COLOR)

# Header
header = tk.Label(root, text="Chatbot Assistant", font=("Arial", 18, "bold"), bg=BG_COLOR, fg=FG_COLOR, pady=10)
header.pack(fill="x")

# Filepath Entry Section
frame_inputs = tk.Frame(root, bg=BG_COLOR)
frame_inputs.pack(pady=10)

file_label = tk.Label(frame_inputs, text="Select JSON File:", font=FONT, bg=BG_COLOR, fg=FG_COLOR)
file_label.grid(row=0, column=0, padx=5, pady=5, sticky="e")

file_entry = tk.Entry(frame_inputs, font=FONT, bg=ENTRY_COLOR, fg=FG_COLOR, bd=1, relief="solid")
file_entry.grid(row=0, column=1, padx=5, pady=5)

# Enable drag-and-drop for file entry
file_entry.drop_target_register(DND_FILES)
file_entry.dnd_bind('<<Drop>>', on_file_drop)

file_button = tk.Button(frame_inputs, text="Browse", command=lambda: file_entry.insert(0, filedialog.askopenfilename(filetypes=[("JSON Files", "*.json")])), bg=BTN_COLOR, fg=FG_COLOR, font=FONT, relief="flat", padx=10)
file_button.grid(row=0, column=2, padx=5, pady=5)

# Session ID Entry
session_label = tk.Label(frame_inputs, text="Enter Session ID:", font=FONT, bg=BG_COLOR, fg=FG_COLOR)
session_label.grid(row=1, column=0, padx=5, pady=5, sticky="e")

session_entry = tk.Entry(frame_inputs, font=FONT, bg=ENTRY_COLOR, fg=FG_COLOR, bd=1, relief="solid")
session_entry.grid(row=1, column=1, padx=5, pady=5)

# Prompt Entry
prompt_label = tk.Label(frame_inputs, text="Enter Prompt:", font=FONT, bg=BG_COLOR, fg=FG_COLOR)
prompt_label.grid(row=2, column=0, padx=5, pady=5, sticky="e")

prompt_entry = tk.Entry(frame_inputs, font=FONT, bg=ENTRY_COLOR, fg=FG_COLOR, bd=1, relief="solid")
prompt_entry.grid(row=2, column=1, padx=5, pady=5)

# Chat History Display
chat_history_text = scrolledtext.ScrolledText(root, width=80, height=20, state='disabled', wrap='word', font=FONT, bg=ENTRY_COLOR, fg=FG_COLOR, bd=1, relief="solid")
chat_history_text.pack(pady=10)

# Buttons Section
frame_buttons = tk.Frame(root, bg=BG_COLOR)
frame_buttons.pack(pady=10)

run_button = tk.Button(frame_buttons, text="Run Chatbot", command=run_chatbot, bg=BTN_COLOR, fg=FG_COLOR, font=FONT, relief="flat", padx=20)
run_button.grid(row=0, column=0, padx=10)

end_button = tk.Button(frame_buttons, text="End Chat", command=end_chat, bg="#FF6347", fg=FG_COLOR, font=FONT, relief="flat", padx=20)
end_button.grid(row=0, column=1, padx=10)

# Mainloop to run the GUI
root.mainloop()
